In [1]:
# Imports
import pandas as pd
import numpy as np
import requests
import joblib
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from datetime import datetime, timedelta

print("✅ Imports réussis!")

✅ Imports réussis!


In [2]:
# Récupération des données historiques (5 ans) avec PLUIE
print("📥 Téléchargement des données historiques (5 ans)...")
print("⏳ Inclut température + précipitations...")

end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=1826)).strftime("%Y-%m-%d")

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 34.0209,
    "longitude": -6.8416,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": "temperature_2m,precipitation,relative_humidity_2m,cloud_cover",
    "timezone": "GMT"
}

response = requests.get(url, params=params)
data = response.json()

print(f"✅ Données récupérées du {start_date} au {end_date}")
print(f"📊 Variables: température, précipitations, humidité, nuages")

📥 Téléchargement des données historiques (5 ans)...
⏳ Inclut température + précipitations...
✅ Données récupérées du 2020-12-10 au 2025-12-10
📊 Variables: température, précipitations, humidité, nuages
✅ Données récupérées du 2020-12-10 au 2025-12-10
📊 Variables: température, précipitations, humidité, nuages


In [3]:
# Préparation des données (Température + Humidité) - Version améliorée
print("🔧 Préparation des données avec features avancées...")

df = pd.DataFrame({
    'date': pd.to_datetime(data['hourly']['time']),
    'temperature': data['hourly']['temperature_2m'],
    'precipitation': data['hourly']['precipitation'],
    'humidity': data['hourly']['relative_humidity_2m'],
    'cloud_cover': data['hourly']['cloud_cover']
})

# Création des features temporelles de base
df['jour_annee'] = df['date'].dt.dayofyear
df['mois'] = df['date'].dt.month
df['heure'] = df['date'].dt.hour
df['jour_mois'] = df['date'].dt.day

# Encodage cyclique (Sin/Cos)
df['day_cos'] = np.cos(2 * np.pi * df['jour_annee'] / 365.25)
df['day_sin'] = np.sin(2 * np.pi * df['jour_annee'] / 365.25)
df['hour_cos'] = np.cos(2 * np.pi * df['heure'] / 24)
df['hour_sin'] = np.sin(2 * np.pi * df['heure'] / 24)
df['month_cos'] = np.cos(2 * np.pi * df['mois'] / 12)
df['month_sin'] = np.sin(2 * np.pi * df['mois'] / 12)

# Features météo existantes
df['cloud_cover_filled'] = df['cloud_cover'].fillna(df['cloud_cover'].mean())

# --- NOUVELLES FEATURES POUR NON-LINÉARITÉ ---
print("   → Ajout de features avancées...")

# 1. Moyennes mobiles (tendances)
df = df.sort_values('date')
df['temp_rolling_24h'] = df['temperature'].rolling(window=24, min_periods=1).mean()
df['temp_rolling_7d'] = df['temperature'].rolling(window=168, min_periods=1).mean()
df['humidity_rolling_24h'] = df['humidity'].rolling(window=24, min_periods=1).mean()

# 2. Interactions entre features
df['hour_month'] = df['heure'] * df['mois']
df['day_hour'] = df['jour_annee'] * df['heure']
df['cloud_hour'] = df['cloud_cover_filled'] * df['heure']

# 3. Features polynomiales
df['jour_annee_sq'] = df['jour_annee'] ** 2
df['heure_sq'] = df['heure'] ** 2
df['mois_sq'] = df['mois'] ** 2

# 4. Indicateurs de période
df['is_winter'] = ((df['mois'] == 12) | (df['mois'] <= 2)).astype(int)
df['is_summer'] = ((df['mois'] >= 6) & (df['mois'] <= 8)).astype(int)
df['is_night'] = ((df['heure'] >= 20) | (df['heure'] <= 6)).astype(int)
df['is_midday'] = ((df['heure'] >= 11) & (df['heure'] <= 15)).astype(int)

# Variable cible pour la pluie
df['pluie'] = (df['precipitation'] > 0).astype(int)

# Suppression des NaN
df = df.dropna()

print(f"✅ {len(df)} observations préparées")
print(f"📊 Features avancées ajoutées pour meilleure précision")
print(f"🌧️ Jours avec pluie: {df['pluie'].sum()} ({df['pluie'].mean()*100:.1f}%)")
df.head()

🔧 Préparation des données avec features avancées...
   → Ajout de features avancées...
✅ 43848 observations préparées
📊 Features avancées ajoutées pour meilleure précision
🌧️ Jours avec pluie: 3844 (8.8%)


,date,temperature,precipitation,humidity,cloud_cover,jour_annee,mois,heure,jour_mois,day_cos,...,day_hour,cloud_hour,jour_annee_sq,heure_sq,mois_sq,is_winter,is_summer,is_night,is_midday,pluie
0,2020-12-10 00:00:00,13.6,0.0,94,43,345,12,0,10,0.939938,...,0,0,119025,0,144,1,0,1,0,0
1,2020-12-10 01:00:00,13.5,0.1,94,32,345,12,1,10,0.939938,...,345,32,119025,1,144,1,0,1,0,1
2,2020-12-10 02:00:00,13.4,0.2,95,59,345,12,2,10,0.939938,...,690,118,119025,4,144,1,0,1,0,1
3,2020-12-10 03:00:00,13.6,0.2,95,67,345,12,3,10,0.939938,...,1035,201,119025,9,144,1,0,1,0,1
4,2020-12-10 04:00:00,13.8,0.2,95,68,345,12,4,10,0.939938,...,1380,272,119025,16,144,1,0,1,0,1


In [4]:
# Entraînement des modèles (Gradient Boosting - Plus précis)
print("🧠 Entraînement des modèles avec Gradient Boosting...")

# Liste complète des features
features = [
    'day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'month_cos', 'month_sin',
    'heure', 'mois', 'jour_annee', 'jour_mois',
    'cloud_cover_filled',
    'temp_rolling_24h', 'temp_rolling_7d', 'humidity_rolling_24h',
    'hour_month', 'day_hour', 'cloud_hour',
    'jour_annee_sq', 'heure_sq', 'mois_sq',
    'is_winter', 'is_summer', 'is_night', 'is_midday'
]

X = df[features]

# 1. Modèle de température (Gradient Boosting)
print("   → Modèle température (Gradient Boosting)...")
y_temp = df['temperature']
model_temp = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    min_samples_split=5,
    min_samples_leaf=2,
    subsample=0.8,
    random_state=42,
    verbose=0
)
model_temp.fit(X, y_temp)

# 2. Modèle d'humidité (Gradient Boosting)
print("   → Modèle humidité (Gradient Boosting)...")
y_humidity = df['humidity']
model_humidity = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    min_samples_split=5,
    min_samples_leaf=2,
    subsample=0.8,
    random_state=42,
    verbose=0
)
model_humidity.fit(X, y_humidity)

print("✅ Les 2 modèles sont entraînés avec Gradient Boosting!")
print(f"📊 Nombre de features utilisées: {len(features)}")

🧠 Entraînement des modèles avec Gradient Boosting...
   → Modèle température (Gradient Boosting)...
   → Modèle humidité (Gradient Boosting)...
   → Modèle humidité (Gradient Boosting)...
✅ Les 2 modèles sont entraînés avec Gradient Boosting!
📊 Nombre de features utilisées: 24
✅ Les 2 modèles sont entraînés avec Gradient Boosting!
📊 Nombre de features utilisées: 24


In [5]:
# Sauvegarde des modèles
import os

save_path = r'C:\Users\User\OneDrive\Bureau\projetDATAMAINING\Prediction-Meteo-Rabat\cerveau_meteo_long_terme.pkl'

# On sauvegarde les 2 modèles ensemble + les données historiques
model_data = {
    'model_temp': model_temp,
    'model_humidity': model_humidity,
    'features': features,
    'historical_data': df  # Sauvegarde des 5 ans de données
}
joblib.dump(model_data, save_path)

print(f"💾 Modèles sauvegardés: {save_path}")
print(f"   → Modèle température ✅")
print(f"   → Modèle humidité ✅")
print(f"   → Données historiques (5 ans) ✅")

if os.path.exists(save_path):
    print("✅ Fichier créé avec succès!")

💾 Modèles sauvegardés: C:\Users\User\OneDrive\Bureau\projetDATAMAINING\Prediction-Meteo-Rabat\cerveau_meteo_long_terme.pkl
   → Modèle température ✅
   → Modèle humidité ✅
   → Données historiques (5 ans) ✅
✅ Fichier créé avec succès!


In [6]:
# Test des modèles
print("🧪 Test des modèles...")

test_data = pd.DataFrame({
    'jour_annee': [datetime.now().timetuple().tm_yday],
    'mois': [datetime.now().month],
    'heure': [14],
    'jour_mois': [datetime.now().day]
})

# Encodage cyclique
test_data['day_cos'] = np.cos(2 * np.pi * test_data['jour_annee'] / 365.25)
test_data['day_sin'] = np.sin(2 * np.pi * test_data['jour_annee'] / 365.25)
test_data['hour_cos'] = np.cos(2 * np.pi * test_data['heure'] / 24)
test_data['hour_sin'] = np.sin(2 * np.pi * test_data['heure'] / 24)
test_data['month_cos'] = np.cos(2 * np.pi * test_data['mois'] / 12)
test_data['month_sin'] = np.sin(2 * np.pi * test_data['mois'] / 12)

# Features météo (valeurs moyennes pour le test)
test_data['cloud_cover_filled'] = df['cloud_cover_filled'].mean()

# Moyennes mobiles (approximation pour le test)
test_data['temp_rolling_24h'] = df['temperature'].tail(24).mean()
test_data['temp_rolling_7d'] = df['temperature'].tail(168).mean()
test_data['humidity_rolling_24h'] = df['humidity'].tail(24).mean()

# Interactions
test_data['hour_month'] = test_data['heure'] * test_data['mois']
test_data['day_hour'] = test_data['jour_annee'] * test_data['heure']
test_data['cloud_hour'] = test_data['cloud_cover_filled'] * test_data['heure']

# Polynomiales
test_data['jour_annee_sq'] = test_data['jour_annee'] ** 2
test_data['heure_sq'] = test_data['heure'] ** 2
test_data['mois_sq'] = test_data['mois'] ** 2

# Indicateurs
test_data['is_winter'] = ((test_data['mois'] == 12) | (test_data['mois'] <= 2)).astype(int)
test_data['is_summer'] = ((test_data['mois'] >= 6) & (test_data['mois'] <= 8)).astype(int)
test_data['is_night'] = ((test_data['heure'] >= 20) | (test_data['heure'] <= 6)).astype(int)
test_data['is_midday'] = ((test_data['heure'] >= 11) & (test_data['heure'] <= 15)).astype(int)

# Prédictions
pred_temp = model_temp.predict(test_data[features])[0]
pred_humidity = model_humidity.predict(test_data[features])[0]

print(f"\n📅 Prédiction pour aujourd'hui à 14h à Rabat:")
print(f"🌡️ Température: {pred_temp:.1f}°C")
print(f"💧 Humidité: {pred_humidity:.0f}%")
print("\n✅ TERMINÉ! Modèle optimisé avec Gradient Boosting et features avancées")
print("🚀 Prédictions plus précises et non-linéaires!")

🧪 Test des modèles...

📅 Prédiction pour aujourd'hui à 14h à Rabat:
🌡️ Température: 18.1°C
💧 Humidité: 61%

✅ TERMINÉ! Modèle optimisé avec Gradient Boosting et features avancées
🚀 Prédictions plus précises et non-linéaires!
